In [8]:
import os
import pandas as pd
from os.path import join as pjoin
import nibabel as nib
import numpy as np

In [15]:
DATA_PATH = './data/'
RUN_NUM = 8
RUN_PROB_NUM = 27
Numerical_IDX = range(18,27)
subList = ['luomanqi']
subjects = ['sub-24']
CLASS_NUM = 9
ATTRIBUTE=[[0,4,5],[1,2,3],[6,7,8]]#edge,obj,symbolic
RULE = [[0,1,6],[2,4,7],[3,5,8]]#const,dist3,prog
problem_sequence=[
    "const_edge_number",
    "const_obj_number",
    "dist3_obj_number",
    "prog_obj_number",
    "dist3_edge_number",
    "prog_edge_number",
    "const_symbolic_number",
    "dist3_symbolic_number",
    "prog_symbolic_number"
]
CLASS_DICT ={
    'attribute':['edge','obj','symbolic'],
    'rule':['const','dist3','prog']
} 
factor_dict ={'const_edge_number':1,'const_obj_number':2,
'const_symbolic_number':3,'dist3_edge_number':4,'dist3_obj_number':5, 
'dist3_symbolic_number':6,'prog_edge_number':7,'prog_obj_number':8,'prog_symbolic_number':9}

In [16]:
def get_txt_filename(subID):#查找是否有补run的情况
    start_of_filename = f'{subList[subID]}run'
    num=0
    for file in os.listdir('trial_type/'):
        if file.startswith(start_of_filename):
            num+=1
    return num

In [17]:
def get_sub_problem_idx(subID):
    
    path = 'trial_type/' + subList[subID] + 'run.txt'
    run_file = open(path, 'r+')

    run_lst = [int(i) for i in list(run_file.readline())[:-1]]
    idx_final = []
    for i in range(RUN_NUM):
        idx_final.append([int(j) for j in run_file.readline().split(',')])
    idx_final = np.array(idx_final)
    time_lst = []
    for i in range(RUN_NUM):
        time_lst.append([int(j) for j in run_file.readline().split(',')])
    time_lst = np.array(time_lst)
    time_lst = time_lst.reshape(RUN_NUM, RUN_PROB_NUM, 2)
    while True:
        line = run_file.readline()
        if line == '':
            break
    if get_txt_filename(subID)==2:#加上补做的run
        path = 'trial_type/' + subList[subID]+'run2.txt'
        run_file2 = open(path, 'r+')

        run_lst2 = [int(i) for i in list(run_file2.readline())[:-1]]
        run_lst2.remove(0)#会将10解析成1，0因此删除0
        run_lst2[run_lst2.index(1)] = 10
        idx_final2 = []
        for i in range(2):
            idx_final2.append([int(j) for j in run_file2.readline().split(',')])
        idx_final2 = np.array(idx_final2)
        time_lst2 = []
        for i in range(2):
            time_lst2.append([int(j) for j in run_file2.readline().split(',')])
        time_lst2 = np.array(time_lst2)
        time_lst2 = time_lst2.reshape(2, RUN_PROB_NUM, 2)
        while True:
            line = run_file2.readline()
            if line == '':
                break
        
        run_lst = run_lst+run_lst2       
        idx_final = np.vstack([idx_final,idx_final2])       
        time_lst = np.vstack([time_lst,time_lst2])
    return run_lst,idx_final,time_lst

In [18]:
def get_one_sub_factors(subID):
    #输出run_num*problem_num*meta_length(一般是8*27)
    run_lst,idx_final,time_lst = get_sub_problem_idx(subID)

    # print(run_lst)#用于检查输出run的次序是否有遗漏
    run_problem_meta = []
    for i in range(len(run_lst)):
        problem_lst = idx_final[run_lst[i]-1]//3
        # print(idx_final[run_lst[i]-1])
        problem_meta = []
        for j in range(len(problem_lst)):
            class_name = problem_sequence[problem_lst[j]]
            class_meta = factor_dict[class_name]
            # print(class_name,class_meta)
            problem_meta.append(class_meta)
        problem_meta=np.array(problem_meta)
        run_problem_meta.append(problem_meta)
    run_problem_meta=np.array(run_problem_meta)
    return run_problem_meta

In [19]:
len(get_one_sub_factors(0))

8

In [20]:
get_one_sub_factors(0)

array([[8, 1, 7, 2, 6, 5, 3, 4, 9, 2, 3, 4, 6, 5, 8, 7, 1, 9, 2, 3, 8, 7,
        1, 4, 5, 6, 9],
       [3, 5, 4, 8, 2, 6, 1, 7, 9, 3, 6, 7, 2, 4, 5, 8, 1, 9, 6, 7, 2, 5,
        1, 4, 3, 8, 9],
       [8, 6, 5, 3, 7, 1, 4, 2, 9, 2, 8, 4, 5, 7, 1, 6, 3, 9, 5, 3, 1, 7,
        2, 6, 8, 4, 9],
       [2, 6, 8, 4, 1, 5, 3, 7, 9, 5, 4, 3, 8, 6, 2, 1, 7, 9, 4, 6, 5, 2,
        3, 7, 1, 8, 9],
       [8, 6, 5, 4, 2, 3, 7, 1, 9, 1, 4, 7, 5, 3, 6, 8, 2, 9, 1, 5, 6, 4,
        2, 8, 3, 7, 9],
       [5, 1, 7, 4, 8, 2, 6, 3, 9, 2, 5, 7, 4, 6, 1, 8, 3, 9, 2, 3, 8, 7,
        5, 6, 4, 1, 9],
       [4, 6, 7, 8, 5, 2, 1, 3, 9, 1, 7, 3, 5, 4, 8, 2, 6, 9, 4, 5, 1, 7,
        3, 8, 2, 6, 9],
       [3, 4, 1, 8, 2, 7, 6, 5, 9, 2, 3, 5, 8, 7, 4, 1, 6, 9, 8, 4, 6, 3,
        1, 2, 5, 7, 9]])

prepare_info

In [15]:
sub_run_info_path = '/nfs/t2/raven/data/code/sub_run_info/'
nifti_path = '/nfs/t2/raven/data/bold/nifti/'
within_sub_path = 'ses-raven/func/'
runs = ['run-01','run-02','run-03','run-04','run-05','run-06','run-07','run-08']
abs_events_paths = []
abs_info_paths = []
sort_abs_info_paths = []

In [16]:
#prepare events file path
for subject in subjects:
    tmp = pjoin(nifti_path,subject)
    tmp = pjoin(tmp,within_sub_path)
    run_paths = []
    for run in runs:
        file_name = '{}_ses-raven_task-action_{}_events.tsv'.format(subject,run)
        run_path = pjoin(tmp,file_name)
        #print(run_path)
        run_paths.append(run_path)
    abs_events_paths.append(run_paths)

In [17]:
#prepare experiment info path
for subject in subjects:
    tmp = pjoin(sub_run_info_path,subject)
    run_paths = []
    for run in os.listdir(tmp):
        run_paths.append(pjoin(tmp,run))
    abs_info_paths.append(run_paths)

In [18]:
for sub in abs_info_paths:
    sub.sort(key=lambda x: int(x[x.find('Run')+3]))
    sort_abs_info_paths.append(sub)      

find factors of each trial

read csv as Dataframe

先做一个被试内，8个run的events读写

In [28]:
# 0 是指第0个被试
for runNum in range(len(sort_abs_info_paths[0])):
    test_info = sort_abs_info_paths[0][runNum]
    #test_info = '/nfs/h1/userhome/liyifan/workingdir/gongzhenxin_run8.csv'
    test_event = abs_events_paths[0][runNum]
    #test_event = '/nfs/t2/RAVEN/data/bold/nifti/sub-02/ses-raven/func/sub-02_ses-raven_task-raven_run-08_events.tsv'
    info = pd.read_csv(test_info)
    trial_start_time = info['problem0.started'][3:30].to_list()
    trial_end_time = info['answer2.stopped'][3:30].to_list()
    scan_start_time = info['key_resp_2.stopped'][2]
    timepoint_list = []
    timepoint_list.extend(trial_start_time)

    onset = []
    for i in range (len(timepoint_list)):
        onsetpoint = timepoint_list[i] - scan_start_time
        onset.append(round(onsetpoint,3))
           
    #add visual stimuli onset
    for i in range (len(timepoint_list)):
        onsetpoint = timepoint_list[i] - scan_start_time
        onset.append(round(onsetpoint,3))

    duration = []
    for i in range(len(trial_start_time)):
        trial_time = trial_end_time[i] - trial_start_time[i]
        duration.append(round(trial_time,3))
        
    # add visual stimuli time  
    for i in range (len(trial_start_time)):
        duration.append(0.1)
            
    # add visual stimuli type
    visual_stimuli_type = np.ones(27,dtype=int)*10

    #trial_type = np.concatenate((get_one_sub_factors(0)[runNum],visual_stimuli_type))

    # this is where we adjust the wrong runs
    if runNum != 0:
        trial_type = np.concatenate((get_one_sub_factors(0)[runNum],visual_stimuli_type))
    if runNum == 0:
        trial_type = np.concatenate((get_one_sub_factors(0)[8],visual_stimuli_type))

    event = pd.read_csv(test_event,sep='\t')
    event = event.drop(columns=['response_time','stim_file','TODO -- fill in rows and add more tab-separated columns if desired'])

    event['onset'] = onset
    event['duration'] = duration
    event['trial_type'] = trial_type

    event.to_csv(test_event,sep='\t',index=False)

KeyError: "['response_time', 'stim_file', 'TODO -- fill in rows and add more tab-separated columns if desired'] not found in axis"